# Algotrading - Aula 7

Simulando Ordens
___


In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from datetime import datetime
import random

# Para fazer o backtesting
from backtesting_v2_1 import TradingSystem, Order, Strategy, Event

Primeiramente vamos criar uma função auxiliar para imprimir o book atual do simulador:

In [2]:
def bookprint(book):
    print('BOOK:')
    if book.bid is not None:
        print('Bid: {}@{}'.format(book.bid.quantity, book.bid.price))
    if book.ask is not None:
        print('Ask: {}@{}'.format(book.ask.quantity, book.ask.price))

Agora uma estratégia dummy só para ver o preenchimento das ordens:

In [3]:
class Dummy(Strategy):
    
    def receive(self, event):
        pass
    
    # Função que captura o preenchimento das ordens enviadas pela estratégia
    # Toda vez que uma order é preenchida, ela é disparada
    def fill(self, id, instrument, price, quantity, status):
        
        print('{} order filled: {}@{}'.format(instrument, quantity, price))        

Desta vez faremos a simulação um pouco diferente, vamos criar um book manualmente ao invés de injetar preços:

In [4]:
instrument = 'Instrumento'

# Criando um Trading System
ts = TradingSystem()

# Criando um Book
ts.createBook(instrument)

# Criando uma Estratégia Genérica
strat = Dummy()

# Criar uma Strategy vazia
ts.subscribe(instrument, strat)

# Imprimindo o book vazio
bookprint(ts.books[instrument])

BOOK:


Injetando manualmente uma oferta de compra e uma oferta de venda no simulador:

In [5]:
# Injetando eventos
ts.inject(Event(instrument, datetime.now, Event.BID, 10, 100)) # BID 100@10
ts.inject(Event(instrument, datetime.now, Event.ASK, 11, 100)) # ASK 100@11

# Imprimindo o book agora com ofertas
bookprint(ts.books[instrument])

BOOK:
Bid: 100@10
Ask: 100@11


Verificando o preenchimento de uma ordem enviada também manualmente:

In [6]:
# Enviando uma compra de 200@MKT
ts.submit(strat.id, Order(instrument, Order.B, 200, 0))

# Imprimindo a posição da estratégia
print(strat._trade.position)

Instrumento order filled: 100@11
{'Instrumento': 100}


Por que a posição é de apenas 100 se a ordem foi de 200?

Possíveis soluções:

- Fatiar a ordem em pequenos pedaços que caibam na fila e não afetem muito o mercado
- Deixar a ordem limitada, esperando que apareçam os compradores
___

Zerando a posição a MKT:

In [7]:
ts.submit(strat.id, Order(instrument, Order.S, strat._trade.position[instrument], 0))
print(strat._trade.position)

Instrumento order filled: -100@10
{}


In [8]:
bookprint(ts.books[instrument])

BOOK:
Bid: 100@10
Ask: 100@11


Enviando então uma ordem limitada no preço de 10.5

In [9]:
ts.submit(strat.id, Order(instrument, Order.B, 100, 10.5))
print(strat._trade.position)

{}


Como percebido, ela não preencheu e ficou pendente, ao contrário da market que foi cancelada. Vamos ver como está o mercado pelos eventos:

In [10]:
# Imprimindo o book de ofertas
bookprint(ts.books[instrument])

BOOK:
Bid: 100@10
Ask: 100@11


Nota: A ordem da estratégia NÃO COMPÕE a fila! Caso contrário ela criaria distorções na simulação que faria com que o Backtesting fosse invalidado.

Mandando um novo evento para a fila, a ordem pendente é preenchida:

In [11]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 10.5, 100))

Instrumento order filled: 100@10.5


___

### Stop Order

Conforme já explicado anteriormente, Stop order tem por objetivo comprar ou vender quando o preço atinge um certo alvo.

Se quiser comprar, o preço alvo precisa ser maior que o preço atual e vice-versa.

Vamos ver como fica a simulação de uma stop order:

In [ ]:
ts = TradingSystem()
ts.createBook(instrument)

class StopPuro(Strategy):
    
    def receive(self, event):
        
        # Se houve um evento de trade
        if event.type == Event.TRADE:
            
            # se o preço do evento for menor que um alvo
            if event.price <= 9.0:
                
                # vende a mercado
                print('Stop Puro disparado!')
                self.submit(self.id, Order(instrument, Order.S, 100, 0))
            
    def fill(self, id, instrument, price, quantity, status):
        
        print('{} order filled: {}@{}'.format(instrument, quantity, price))

        
stop = StopPuro()
ts.subscribe(instrument, stop)

In [ ]:
# Formação do mercado:

ts.inject(Event(instrument, datetime.now, Event.BID, 9, 100))
ts.inject(Event(instrument, datetime.now, Event.ASK, 10, 100))

bookprint(ts.books[instrument])

In [ ]:
# Simula um evento de trade no preço alvo do stop

ts.inject(Event(instrument, datetime.now, Event.TRADE, 9, 100))

Problema: Uma ordem de stop sofre dos mesmos problemas de uma ordem a mercado

- Compra/Vende no preço que estiver na fila
- Não preenche tudo se a fila não suportar a quantidade

Vamos simular:

In [ ]:
# Nesse exemplo saiu um trade no preço alvo (9), mas a fila ficou limpa e a próxima compra está apenas no 7:

ts.inject(Event(instrument, datetime.now, Event.BID, 7, 100))
ts.inject(Event(instrument, datetime.now, Event.TRADE, 9, 100))

Não pareceu um bom negócio, vendeu a 7 o que deveria ter vendido a 9.

Vamos analisar algumas soluções:

#### Solução 1: Stop Limit

É análogo ao Stop, mas envia uma ordem limitada.

In [ ]:
ts = TradingSystem()
ts.createBook(instrument)

class StopLimit(Strategy):
    
    def receive(self, event):
        if event.type == Event.TRADE:
            if event.price <= 8.5:
                print('Stop Limit disparado!')
                self.submit(self.id, Order(instrument, Order.SS, 100, 8.5))
            
    def fill(self, id, instrument, price, quantity, status):
        
        print('{} order filled: {}@{}'.format(instrument, quantity, price))

        
stoplmt = StopLimit()
ts.subscribe(instrument, stoplmt)

Situação idealizada: quando sai um trade, imagina-se que sobram ofertas ainda no preço alvo

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.BID, 8.5, 200))
ts.inject(Event(instrument, datetime.now, Event.TRADE, 8.5, 100))

Na prática: um stop muito longo é disparado quando o mercado de move muito rapidamente, deixando a fila com buracos:

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.BID, 8, 100))
ts.inject(Event(instrument, datetime.now, Event.TRADE, 8.5, 100))

Nota que a situação acima pode ser pior que a do stop normal. Se o mercado continuar caindo, vai continuar exposto e não ter fechado a posição, tendo que zerar a carteira em um preço pior ainda.
___

Caso o mercado seja **bondoso** e volte, a ordem preenche:

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.BID, 8.5, 100))

Existe ainda uma situação tão complicada quanto ao vácuo acima. Muitas vezes a fila flutua sem haver operações. Por exemplo o bid o ask vão abaixo consistentemente e nenhuma operação é registrada. Quando acontecer algum trade, pode ser que seja tarde demais para agir. Pense sobre isso.

___

#### Solução 2: Stop com trigger no bid ou ask

Ao passo que o stop convencional está disponível abundantemente em plataformas, esse tipo de stop é mais preciso, porém normalmente é preciso programar sua própria ordem

A única diferença é que não é preciso esperar acontecer uma operação no preço alvo para executar o stop, se o mercado espremer na fila, não espera a catástrofe, saia.

In [ ]:
ts = TradingSystem()
ts.createBook(instrument)

class StopLimitv2(Strategy):
    
    def receive(self, event):
        
        # Análogo ao anterior, mas olha para o BID e não o TRADE
        if event.type == Event.BID:
            
            if event.price <= 9.0:
                print('Stop Limit v2 disparado!')
                self.submit(self.id, Order(instrument, Order.SS, 100, event.price))
            
    def fill(self, id, instrument, price, quantity, status):
        
        print('{} order filled: {}@{}'.format(instrument, quantity, price))

        
stoplmt2 = StopLimitv2()
ts.subscribe(instrument, stoplmt2)

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.BID, 9.2, 100))

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.BID, 9.1, 100))

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.BID, 9.0, 100))

Parece bom, contudo ainda tem problemas:

* Não garante transição suave e sem saltos, poderia ir para o 8 diretamente em dia de mercado volátil.
* A quantidade pode ser insuficiente
___

### Stop móvel

Uma outra forma de realizar um stop, ou até realizar uma alocação é utilizar um tipo de stop em que o preço alvo flutua ao favor da estratégia. Veja o exemplo:

- A estratégia está zerada.
- A estratégia quer comprar. Se mandar uma ordem, será no preço que está na fila.
- E se ao invés de comprar, coloca-se um alvo: preço atual mais um delta, se bater no alvo, compra-se.
- Se o preço atual cair, ajusta-se um novo alvo, mais baixo, permitindo comprar em um preço melhor!

Vamos ver na prática:


In [ ]:
ts = TradingSystem()
ts.createBook(instrument)

class MovingStop(Strategy):
    def __init__(self):
        
        # Preço que desejo comprar
        self.trigger = 9
        
        # Preço alvo, ainda não disparado
        self.target = None
    
    def receive(self, event):
        if event.type == Event.ASK:
            
            # Se o alvo ainda não foi disparado e bater no trigger:
            if self.target is None and event.price <= self.trigger:
                
                print('Moving Stop disparado!')
                
                # Esse é o novo alvo + um delta
                self.target = event.price
                print('Preço stop: {}'.format(self.target + 0.5))
            
            # Se o alvo já está sendo rastreado
            if self.target is not None:
                
                # Se o preço voltar (subir), compro
                if event.price >= self.target + 0.5:
                    print('Preço stop atingido, comprando')
                    self.submit(self.id, Order(instrument, Order.B, 100, 0))

                # Se o preço cair, traço novas metas
                elif event.price < self.target:
                    
                    # Atualização do target
                    self.target = event.price
                    print('Novo preço stop: {}'.format(self.target + 0.5))
                
            
    def fill(self, id, instrument, price, quantity, status):
        
        print('{} order filled: {}@{}'.format(instrument, quantity, price))

        
movestop = MovingStop()
ts.subscribe(instrument, movestop)

Simulando passo a passo:

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 9.2, 100))

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 9.1, 100))

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 9.0, 100))

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 8.9, 100))

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 8.5, 100))

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 8.0, 100))

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 8.5, 100))

Pergunta: Qual é o tamanho ideal do delta?

- Se ele for muito grande, quando voltar, a estratégia terá comprado em um preço ruim (ou não terá comprado)
- Se ele for muito pequeno, qualquer pequena flutuação será suficiente para disparar a ordem

### Outras Estratégias: OCO - One cancels other

Muito utilizado para realizar stop gain e stop loss de forma autonôma sem intervenção humana.

Dispara-se duas ordens, uma limitada (gain) e uma stop (loss). Caso alguma ordem seja executada, cancela-se a outra.

In [ ]:
ts = TradingSystem()
ts.createBook(instrument)

class OCO(Strategy):
    
    def __init__(self):
        self.side = 0
        self.target = None
        self.gain_id = 0
    
    def receive(self, event):
        
        # se a posição está zerada, compra 100
        if self.side == 0:
            self.submit(self.id, Order(instrument, Order.B, 100, 0))
        
        
        # Se houve um evento de trade
        if event.type == Event.BID:
            
            # se o preço do evento for menor que um alvo
            if self.target is not None and event.price <= self.target:
                
                # vende a mercado
                print('Stop disparado!')
                self.submit(self.id, Order(instrument, Order.S, 100, 0))
            
    def fill(self, id, instrument, price, quantity, status):
        
        print('{} order filled: {}@{}'.format(instrument, quantity, price))
        
        # se eu estou zerado
        if self.side == 0: 
            # ordem de stop gain limitada
            order = Order(instrument, Order.S, quantity, price + 1)
            self.submit(self.id, order)
            # guardo para o controle da ordem de stop gain
            self.gain_id = order.id
            # guardo a meta para o stop loss
            self.target = price - 1
            print('montei posição e disparei os stops!')
            
            self.side = 1
            
        else: # estou posicionado
            
            # invalido as ordens de stop
            self.cancel(self.id, self.gain_id)
            self.target = None
            
            print('ordens de stop desarmadas')
        
oco = OCO()
ts.subscribe(instrument, oco)

In [ ]:
ts.inject(Event(instrument, datetime.now, Event.ASK, 10.0, 100))

In [ ]:
# Stop Gain
ts.inject(Event(instrument, datetime.now, Event.BID, 11, 100))

In [ ]:
# Stop loss desmontado, nada a ser feito

ts.inject(Event(instrument, datetime.now, Event.BID, 9, 100))

In [ ]:
# Rearmando a estratégia:

ts.inject(Event(instrument, datetime.now, Event.BID, 9.9, 100))

# alocando de novo
oco.side = 0
ts.inject(Event(instrument, datetime.now, Event.ASK, 10.0, 100))

In [ ]:
# Stop Loss
ts.inject(Event(instrument, datetime.now, Event.BID, 9, 100))

In [ ]:
# Stop gain desmontado, nada a ser feito
ts.inject(Event(instrument, datetime.now, Event.BID, 11, 100))

Vocês podem combinar diversas formas para criar ordens sintéticas das mais variadas, incluindo o tempo como um fato para ordens do to VWAP, etc.

Você pode combinar modelos de execução com modelos direcionais para realizar um backtesting de uma estratégia mais ampla.